In [1]:
# from car_detection.models import Darknet
import sys
sys.path.append("..")

from roadmap_segmentation.pix2vox import pix2vox
from self_supervised.model import CPCModel
from self_supervised.config import Args

In [2]:
class DockSLL():

    def __init__(self, sl_model_addr='../model_weights/best-model-pix2vox.pth', ssl_model_addr = '../model_weights/best-model-cpc.pth'):
 
        self.use_cuda = torch.cuda.is_available()
        self.device = torch.device('cuda' if self.use_cuda else 'cpu')
        
        self.ssl_model = CPCModel(Args())
        if self.use_cuda:
            state_dict1 = torch.load(ssl_model_addr)
            self.ssl_model.cuda()
            self.ssl_model.load_state_dict(state_dict1)  
        self.ssl_model.eval()
        
        
        self.sl_model = pix2vox()
        if self.use_cuda:
            state_dict2 = torch.load(sl_model_addr)
            self.sl_model.cuda()
            self.sl_model.load_state_dict(state_dict2)  
        self.sl_model.eval()
        
        
    def copy_ssl_encoder(self):
        
        for param_q, param_k in zip(self.ssl_model.encoder_q.parameters(), self.sl_model.encoder.parameters()):
            try:
                param_k.data.copy_(param_q.data)
                param_k.requires_grad = False
            except: 
                print('copied ssl encoder to the lowest layer, weight freezed')
                break
       
    def get_model_from_ssl(self):
        self.copy_ssl_encoder()
        return self.sl_model
        
        


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

pix2vox = DockSLL().get_model_from_ssl()

Feature extractor: resnet50
Feature extractor: resnet50
RNN model: LSTM
RNN model: LSTM
Running on: cuda, Tesla K80
copied ssl encoder to the lowest layer, weight freezed
